In [12]:
%matplotlib inline
import sys, os, time
import numpy as np
import pandas as pd
import collections
import pickle

import matplotlib.pyplot as plt

In [13]:
def blockshaped(arr, nrows, ncols):
    """
    From: https://stackoverflow.com/questions/16856788/slice-2d-array-into-smaller-2d-arrays
    
    Return an array of shape (n, nrows, ncols) where
    n * nrows * ncols = arr.size

    If arr is a 2D array, the returned array should look like n subblocks with
    each subblock preserving the "physical" layout of arr.
    """
    h, w = arr.shape
    assert h % nrows == 0, "{} rows is not evenly divisble by {}".format(h, nrows)
    assert w % ncols == 0, "{} cols is not evenly divisble by {}".format(w, ncols)
    return (arr.reshape(h//nrows, nrows, -1, ncols)
               .swapaxes(1,2)
               .reshape(-1, nrows, ncols))

In [3]:
NLCD_CLASSES  = collections.OrderedDict([
    (11, ("Open Water", "areas of open water, generally with less than 25% cover of vegetation or soil.")),
    (12, ("Perennial Ice/Snow", "areas characterized by a perennial cover of ice and/or snow, generally greater than 25% of total cover.")),
    (21, ("Developed, Open Space", "areas with a mixture of some constructed materials, but mostly vegetation in the form of lawn grasses. Impervious surfaces account for less than 20% of total cover. These areas most commonly include large-lot single-family housing units, parks, golf courses, and vegetation planted in developed settings for recreation, erosion control, or aesthetic purposes.")),
    (22, ("Developed, Low Intensity", "areas with a mixture of constructed materials and vegetation. Impervious surfaces account for 20% to 49% percent of total cover. These areas most commonly include single-family housing units.")),
    (23, ("Developed, Medium Intensity", "areas with a mixture of constructed materials and vegetation. Impervious surfaces account for 50% to 79% of the total cover. These areas most commonly include single-family housing units.")),
    (24, ("Developed High Intensity", "highly developed areas where people reside or work in high numbers. Examples include apartment complexes, row houses and commercial/industrial. Impervious surfaces account for 80% to 100% of the total cover.")),
    (31, ("Barren Land (Rock/Sand/Clay)", "areas of bedrock, desert pavement, scarps, talus, slides, volcanic material, glacial debris, sand dunes, strip mines, gravel pits and other accumulations of earthen material. Generally, vegetation accounts for less than 15% of total cover.")),
    (41, ("Deciduous Forest", "areas dominated by trees generally greater than 5 meters tall, and greater than 20% of total vegetation cover. More than 75% of the tree species shed foliage simultaneously in response to seasonal change.")),
    (42, ("Evergreen Forest", "areas dominated by trees generally greater than 5 meters tall, and greater than 20% of total vegetation cover. More than 75% of the tree species maintain their leaves all year. Canopy is never without green foliage.")),
    (43, ("Mixed Forest", "areas dominated by trees generally greater than 5 meters tall, and greater than 20% of total vegetation cover. Neither deciduous nor evergreen species are greater than 75% of total tree cover.")),
    (52, ("Shrub/Scrub", "areas dominated by shrubs; less than 5 meters tall with shrub canopy typically greater than 20% of total vegetation. This class includes true shrubs, young trees in an early successional stage or trees stunted from environmental conditions.")),
    (71, ("Grassland/Herbaceous", "areas dominated by gramanoid or herbaceous vegetation, generally greater than 80% of total vegetation. These areas are not subject to intensive management such as tilling, but can be utilized for grazing.")),
    (81, ("Pasture/Hay", "areas of grasses, legumes, or grass-legume mixtures planted for livestock grazing or the production of seed or hay crops, typically on a perennial cycle. Pasture/hay vegetation accounts for greater than 20% of total vegetation.")),
    (82, ("Cultivated Crops", "areas used for the production of annual crops, such as corn, soybeans, vegetables, tobacco, and cotton, and also perennial woody crops such as orchards and vineyards. Crop vegetation accounts for greater than 20% of total vegetation. This class also includes all land being actively tilled.")),
    (90, ("Woody Wetlands", "areas where forest or shrubland vegetation accounts for greater than 20% of vegetative cover and the soil or substrate is periodically saturated with or covered with water.")),
    (95, ("Emergent Herbaceous Wetlands", "Areas where perennial herbaceous vegetation accounts for greater than 80% of vegetative cover and the soil or substrate is periodically saturated with or covered with water."))
])

In [4]:
DATA_DIR = "/home/caleb/data/"

In [5]:
split_names = [
    "de_1m_2013",
    "ny_1m_2013",
    "md_1m_2013",
    "pa_1m_2013",
    "va_1m_2014",
    "wv_1m_2014"
]

In [ ]:
for split_name in split_names:
    df = pd.read_csv(DATA_DIR + "%s_extended-train_patches.csv" % (split_name))
    fns = (DATA_DIR + df["patch_fn"]).values
    
    per_nlcd_counts = collections.defaultdict(list)

    for i, fn in enumerate(fns):
        with np.load(fn) as f:
            data = f["arr_0"].squeeze()
            data = np.rollaxis(data, 0, 3)

            data_lc = data[:,:,8]
            data_lc[data_lc == 5] = 4
            data_lc[data_lc == 6] = 4

            data_nlcd = data[:,:,9]

            # break up into 32x32 blocks
            data_lc = blockshaped(data_lc, 32, 32).reshape(64, 32*32)
            data_nlcd = blockshaped(data_nlcd, 32, 32).reshape(64, 32*32)

            # iterate through each block
            for j in range(64):
                nlcd_vals, nlcd_counts = np.unique(data_nlcd[j], return_counts=True)
                nlcd_counts = nlcd_counts / (32.0* 32.0)
                max_count_idx = np.argmax(nlcd_counts)
                max_count = nlcd_counts[max_count_idx]
                nlcd_val = int(nlcd_vals[max_count_idx])

                if max_count > 0.9: # the most frequently occuring NLCD class covers over 90% of the current 32x32 window
                    mask = data_nlcd[j] == nlcd_val
                    lc_vals, lc_counts = np.unique(data_lc[j,mask], return_counts=True)
                    lc_counts_flat = np.zeros((5), dtype=int)
                    for val, count in zip(lc_vals, lc_counts):
                        lc_counts_flat[int(val)] += count
                    per_nlcd_counts[nlcd_val].append(lc_counts_flat)
        
    with open("per_nlcd_counts_%s.p" % (split_name), "wb") as f:
        pickle.dump(per_nlcd_counts, f, protocol=pickle.HIGHEST_PROTOCOL)
    
    print(split_name)
    for nlcd_val, (nlcd_class, nlcd_description) in NLCD_CLASSES.items():

        if nlcd_val in per_nlcd_counts:
            counts_per_block = per_nlcd_counts[nlcd_val]

            lc_vals = np.stack(counts_per_block, axis=1).T
            lc_vals = lc_vals[:,1:]
            lc_vals = lc_vals / lc_vals.sum(axis=1, keepdims=True)

            lc_means = lc_vals.mean(axis=0)
            lc_stdevs = lc_vals.std(axis=0)
            
            print("%d|%s|%0.4f +/- %0.4f|%0.4f +/- %0.4f|%0.4f +/- %0.4f|%0.4f +/- %0.4f|" % (
                nlcd_val, nlcd_class,
                lc_means[0], lc_stdevs[0],
                lc_means[1], lc_stdevs[1],
                lc_means[2], lc_stdevs[2],
                lc_means[3], lc_stdevs[3],
            ))

        else:
            print("%d|%s|||||" % (nlcd_val, nlcd_class))
    print("")
    print("")

## Scratch

In [6]:
per_nlcd_counts = collections.defaultdict(list)

for i, fn in enumerate(fns):
    if i % 100 == 0:
        print(i, len(fns))
    with np.load(fn) as f:
        data = f["arr_0"].squeeze()
        data = np.rollaxis(data, 0, 3)

        data_lc = data[:,:,8]
        data_lc[data_lc == 5] = 4
        data_lc[data_lc == 6] = 4

        data_nlcd = data[:,:,9]

        # break up into 32x32 blocks
        data_lc = blockshaped(data_lc, 32, 32).reshape(64, 32*32)
        data_nlcd = blockshaped(data_nlcd, 32, 32).reshape(64, 32*32)
        
        # iterate through each block
        for j in range(64):
            nlcd_vals, nlcd_counts = np.unique(data_nlcd[j], return_counts=True)
            nlcd_counts = nlcd_counts / (32.0* 32.0)
            max_count_idx = np.argmax(nlcd_counts)
            max_count = nlcd_counts[max_count_idx]
            nlcd_val = int(nlcd_vals[max_count_idx])
            
            if max_count > 0.9: # the most frequently occuring NLCD class covers over 90% of the current 32x32 window
                mask = data_nlcd[j] == nlcd_val
                lc_vals, lc_counts = np.unique(data_lc[j,mask], return_counts=True)
                lc_counts_flat = np.zeros((5), dtype=int)
                for val, count in zip(lc_vals, lc_counts):
                    lc_counts_flat[int(val)] += count
                per_nlcd_counts[nlcd_val].append(lc_counts_flat)

0 2500
100 2500
200 2500
300 2500
400 2500
500 2500
600 2500
700 2500
800 2500
900 2500
1000 2500
1100 2500
1200 2500
1300 2500
1400 2500
1500 2500
1600 2500
1700 2500
1800 2500
1900 2500
2000 2500
2100 2500
2200 2500
2300 2500
2400 2500


In [28]:
print("de_1m_2013")
for nlcd_val, (nlcd_class, nlcd_description) in NLCD_CLASSES.items():
    
    if nlcd_val in per_nlcd_counts:
        counts_per_block = per_nlcd_counts[nlcd_val]
        
        lc_vals = np.stack(counts_per_block, axis=1).T
        lc_vals = lc_vals[:,1:]
        lc_vals = lc_vals / lc_vals.sum(axis=1, keepdims=True)

        lc_means = lc_vals.mean(axis=0)
        lc_stdevs = lc_vals.std(axis=0)
        
        print("%d,%s,%0.4f +/- %0.4f,%0.4f +/- %0.4f,%0.4f +/- %0.4f,%0.4f +/- %0.4f" % (
            nlcd_val, nlcd_class,
            lc_means[0], lc_stdevs[0],
            lc_means[1], lc_stdevs[1],
            lc_means[2], lc_stdevs[2],
            lc_means[3], lc_stdevs[3],
        ))
        
    else:
        print("%d,%s,,,,," % (nlcd_val, nlcd_class))

de_1m_2013
11,Open Water,0.9426 +/- 0.1919,0.0168 +/- 0.0955,0.0395 +/- 0.1646,0.0012 +/- 0.0187
12,Perennial Ice/Snow,,,,,
21,Developed, Open Space,0.0076 +/- 0.0656,0.2095 +/- 0.3086,0.6790 +/- 0.3297,0.1039 +/- 0.1560
22,Developed, Low Intensity,0.0231 +/- 0.1118,0.2435 +/- 0.2520,0.3922 +/- 0.2460,0.3411 +/- 0.1920
23,Developed, Medium Intensity,0.0250 +/- 0.1070,0.1054 +/- 0.1567,0.2930 +/- 0.2187,0.5766 +/- 0.2453
24,Developed High Intensity,0.0050 +/- 0.0560,0.0129 +/- 0.0432,0.0790 +/- 0.1458,0.9031 +/- 0.1704
31,Barren Land (Rock/Sand/Clay) ,0.0243 +/- 0.1220,0.0712 +/- 0.1554,0.2678 +/- 0.3691,0.6368 +/- 0.4425
41,Deciduous Forest,0.0036 +/- 0.0476,0.9215 +/- 0.2133,0.0578 +/- 0.1692,0.0171 +/- 0.0710
42,Evergreen Forest,0.0000 +/- 0.0007,0.9779 +/- 0.0951,0.0144 +/- 0.0694,0.0077 +/- 0.0445
43,Mixed Forest,0.0001 +/- 0.0023,0.9680 +/- 0.1271,0.0225 +/- 0.0855,0.0094 +/- 0.0553
52,Shrub/Scrub,0.0215 +/- 0.0983,0.7449 +/- 0.3442,0.1802 +/- 0.2732,0.0533 +/- 0.1461
71,Grassland